# Single-site latent localized regression on Prevent Data

In [83]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import itertools

from torch.utils.tensorboard import SummaryWriter

# Load own module for data loading and training
import sys
sys.path.append('../../src/methods')
from single_site_ae_loreg_module import create_data_loader, initialize_training, train_model

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [84]:
# Set random seed for reproducibility
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # If using CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

## Load and Prepare data

In [ ]:
prevent = pd.read_csv('../../data/processed/prevent_num_imp_varL2_std.csv')

In [ ]:
single_site_data = prevent

# Drop unnecessary columns
single_site_data = single_site_data.drop(columns=['SGRQ_symptoms_W', 
                                                  'SGRQ_activity_W', 
                                                  'SGRQ_impacts_W', 
                                                  'CAT_Score_W'])

# Get SGRQ_total_W from respective next visit to predict based on previous visit
single_site_data['SGRQ_total_shift'] = single_site_data.groupby(['PID'])['SGRQ_total_W'].shift(-1)

# Remove original `SGRQ_total_W` column
single_site_data = single_site_data.drop(columns=["SGRQ_total_W"])

# Remove NAs created in `SGRQ_total_shift`
single_site_data = single_site_data.dropna()

# Filter vor only Baseline visit
single_site_data = single_site_data[single_site_data['VISIT']=="Baseline (V0)"]

# Drop unneeded columns: `CENTER`, `PID`, `VISIT`
single_site_data = single_site_data.drop(columns=['CENTER', 'PID', 'VISIT'])

# Remove outliers
Q1 = single_site_data.quantile(0.25)
Q3 = single_site_data.quantile(0.75)
IQR = Q3 - Q1

single_site_data = single_site_data[~((single_site_data < (Q1 - 4 * IQR)) | (single_site_data > (Q3 + 4 * IQR))).any(axis=1)]

# Reset index
single_site_data = single_site_data.reset_index(drop=True)

# Save processed data
# single_site_data.to_csv('../../data/processed/prevent_num_imp_varL2_std_single_site.csv', index=False)

# Print size of the single-site data
single_site_size = len(single_site_data)
print(single_site_size)

# Normalize each column
single_site_data = (single_site_data - single_site_data.mean()) / single_site_data.std()

from sklearn.model_selection import train_test_split
# Split data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(single_site_data, test_size=0.2, random_state=123)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

test_data_later = test_data.copy()

print(len(train_data), len(test_data))

# Save processed data
single_site_data.to_csv('../../data/processed/prevent_num_imp_varL2_std_single_site.csv', index=False)

## Set Up Model

In [ ]:
# Define hyperparameters and settings
input_size = single_site_data.shape[1] - 1
latent_size = 4
hidden_size_1 = latent_size**3
hidden_size_2 = latent_size**2
target_size = single_site_data.shape[1] - 1
batch_size = 1
num_epochs = 200
log_interval = 2
learning_rate = 1e-4
weight_decay = 1e-4
alpha_values = [1]
theta_values = [0.021]
gamma_values = [1e-4]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kernel = 'gaussian'
num_batches = 3
sigma = 1.2
k_nearest = 0.3

# Main training loop
tune_iter = 1
loss_values = []


for alpha, theta, gamma in itertools.product(alpha_values, theta_values, gamma_values):
    # dynamic log dir for multiple runs
    log_dir = f"../../runs/alpha{alpha}_theta{theta}_gamma{gamma}_sigma{sigma}_weightdecay{weight_decay}_k_nearest{k_nearest}_batch{num_batches}_latent{latent_size}_epochs{num_epochs}"
    writer = SummaryWriter(log_dir=log_dir) 

    print(f"Tuning parameter combination {tune_iter} with alpha={alpha}, theta={theta}, gamma={gamma}")
    tune_iter += 1

    # Initialize model and optimizer
    model, optimizer, loss_values = initialize_training(input_size, 
                                                        hidden_size_1, 
                                                        hidden_size_2, 
                                                        latent_size, 
                                                        device, 
                                                        learning_rate, 
                                                        weight_decay)

    # Train the model
    current_loss_values = train_model(train_data,"SGRQ_total_shift",batch_size, 1000, num_batches, writer, model, optimizer, num_epochs, log_interval, alpha, theta, gamma, device, sigma, k_nearest, kernel)
    loss_values.append(current_loss_values)

writer.close()


## Save away the data

In [ ]:

data_loader = create_data_loader(train_data, "SGRQ_total_shift", batch_size, num_batches, shuffle=False)
latent_data = model.encoder(data_loader.dataset.data_x).detach().numpy()

# Reconstruction training
reconstruction_train = model.decoder(torch.tensor(latent_data).float()).detach().numpy()
reconstruction_train = pd.DataFrame(reconstruction_train, columns=train_data.columns[:-1])

# Reconstruction loss
reconstruction_loss_train = np.mean((train_data - reconstruction_train)**2)
print(f"Reconstruction loss: {reconstruction_loss_train}")

latent_data = pd.DataFrame(latent_data, columns=[f'Latent{i}' for i in range(latent_size)])

# Merge with y column
latent_data['Y'] = train_data["SGRQ_total_shift"]
latent_data.head()

print(len(data_loader.dataset.data_x), len(latent_data))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_latent_spaces(latent_df, latent_size):
    # Calculate the number of rows and columns for subplots
    n_cols = 2
    n_rows = (latent_size + 1) // n_cols

    # Create a figure and set of subplots
    plt.figure(figsize=(n_cols * 5, n_rows * 5))
    
    for i in range(latent_size):
        plt.subplot(n_rows, n_cols, i + 1)
        sns.scatterplot(data=latent_df, x=f'Latent{i}', y='Y', alpha=0.5)
        plt.title(f'Latent Space {i} vs Y of AE with Regression')
    
    plt.tight_layout()
    plt.show()

# Plot latent spaces
plot_latent_spaces(latent_data, latent_size)


In [ ]:
single_site_data = prevent

# Drop unnecessary columns
single_site_data = single_site_data.drop(columns=['SGRQ_symptoms_W', 
                                                  'SGRQ_activity_W', 
                                                  'SGRQ_impacts_W', 
                                                  'CAT_Score_W'])

# Get SGRQ_total_W from respective next visit to predict based on previous visit
single_site_data['SGRQ_total_shift'] = single_site_data.groupby(['PID'])['SGRQ_total_W'].shift(-1)

# Remove original `SGRQ_total_W` column
single_site_data = single_site_data.drop(columns=["SGRQ_total_W"])

# Remove NAs created in `SGRQ_total_shift`
single_site_data = single_site_data.dropna()

# Filter vor only Baseline visit
# single_site_data = single_site_data[single_site_data['VISIT'].isin(['Baseline (V0)', 'Visit 5 (SV5)'])]
single_site_data = single_site_data[single_site_data['VISIT']=="Baseline (V0)"]

# Drop unneeded columns: `CENTER`, `PID`, `VISIT`
single_site_data = single_site_data.drop(columns=['CENTER', 'PID', 'VISIT'])

# Remove outliers
Q1 = single_site_data.quantile(0.25)
Q3 = single_site_data.quantile(0.75)
IQR = Q3 - Q1

single_site_data = single_site_data[~((single_site_data < (Q1 - 4 * IQR)) | (single_site_data > (Q3 + 4 * IQR))).any(axis=1)]

# Normalize each column
single_site_data = (single_site_data - single_site_data.mean()) / single_site_data.std()

# Reset index
single_site_data = single_site_data.reset_index(drop=True)

train_data, test_data = train_test_split(single_site_data, test_size=0.2, random_state=123)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Merge with latent data
train_latent = pd.concat([train_data, latent_data], axis=1)

# Drop outcome
train_latent['Y'] = train_latent['SGRQ_total_shift']
train_latent = train_latent.drop(columns=['SGRQ_total_shift'])

print(len(train_data), len(train_latent))

# Save processed data
# train_latent.to_csv(f'../../results/tables/singlesite/train_merged_latent_alpha{alpha}_theta{theta}_gamma{gamma}_sigma{sigma}_weightdecay{weight_decay}_k_nearest{k_nearest}_batch{num_batches}_latent{latent_size}_epochs{num_epochs}.csv', index=False)


## Prepare test data

In [ ]:
data_loader = create_data_loader(test_data, "SGRQ_total_shift", batch_size, num_batches, shuffle=False)
latent_data = model.encoder(data_loader.dataset.data_x).detach().numpy()

reconstructed_test = model.decoder(torch.tensor(latent_data).float()).detach().numpy()
reconstructed_test = pd.DataFrame(reconstructed_test, columns=test_data.columns[:-1])

# Reconstruction loss
reconstruction_loss = np.mean((test_data - reconstructed_test)**2)
print(f"Reconstruction loss: {reconstruction_loss}")

latent_data = pd.DataFrame(latent_data, columns=[f'Latent{i}' for i in range(latent_size)])

# Merge with y column
latent_data['Y'] = test_data["SGRQ_total_shift"]
latent_data.head()

test_latent = pd.concat([test_data, latent_data], axis=1)
test_latent = test_latent.drop(columns=['SGRQ_total_shift'])

print(len(test_latent))
print(len(train_latent))

# Merge train and test data
train_latent['train'] = 1
test_latent['train'] = 0

# Row bind train and test data
merged_latent = pd.concat([train_latent, test_latent], axis=0)

merged_latent.to_csv(f'../../results/tables/singlesite/traintest_latent_alpha{alpha}_theta{theta}_gamma{gamma}_sigma{sigma}_weightdecay{weight_decay}_k_nearest{k_nearest}_batch{num_batches}_latent{latent_size}_epochs{num_epochs}.csv', index=False)
